In [1]:
pip install spotipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 17.3 MB/s eta 0:00:00


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [7]:
def scrape_spotify_custom_format(playlist_url, era_label):
    # Ambil URI Playlist
    playlist_uri = playlist_url.split("/")[-1].split("?")[0]

    # Wadah data
    raw_tracks = []
    artist_ids_to_fetch = set()

    # 1. Ambil semua lagu dari playlist
    print(f"Sedang membaca playlist {era_label}...")
    results = sp.playlist_tracks(playlist_uri)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    # 2. Kumpulkan ID Artis untuk mengambil Genre nanti
    for item in tracks:
        if item['track'] and item['track']['artists']:
            artist_ids_to_fetch.add(item['track']['artists'][0]['id'])

    # 3. Ambil Genre Artis (Batch Processing)
    print(f"Mengambil data genre untuk {len(artist_ids_to_fetch)} artis...")
    genre_map = get_artist_genres_batch(list(artist_ids_to_fetch))

    # 4. Susun Data Sesuai Format Excel
    final_data = []

    for item in tracks:
        track = item['track']
        if not track: continue

        # Data Extraction
        judul = track['name']
        artis_utama = track['artists'][0]
        nama_artis = artis_utama['name']
        artist_id = artis_utama['id']

        # Ambil tahun rilis (Format YYYY)
        try:
            tahun = track['album']['release_date'][:4]
        except:
            tahun = "0000"

        # Ambil Genre dari map yang sudah kita buat
        genre = genre_map.get(artist_id, "Pop")

        # Struktur Data Sesuai Request
        # Judul, Artis, Tahun, Era, Genre, Penulis, Lirik
        row = {
            'Judul': judul,
            'Artis': nama_artis,
            'Tahun': tahun,
            'Era': era_label,
            'Genre': genre,
            'Penulis': nama_artis, # Placeholder: API Spotify tidak menyediakan Songwriter
            'Lirik': ""            # Dikosongkan sesuai request
        }
        final_data.append(row)

    return final_data

### 1. Konfigurasi Kredensial Spotify

Bagian ini berisi pengaturan untuk menghubungkan ke API Spotify menggunakan Client ID dan Client Secret Anda. Pastikan untuk mengganti placeholder dengan kredensial asli Anda dari [Spotify Developer Dashboard](https://developer.spotify.com/dashboard/).

In [3]:
# Masukkan Client ID dan Secret dari dashboard developer spotify kamu
CLIENT_ID = '910ef7d9cd374024bb13aebdb72935f9'
CLIENT_SECRET = '7d63adaab61840eca3ca8581ec1f407a'

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

### 2. Fungsi Utama untuk Scraping Data

Berikut adalah dua fungsi utama yang digunakan:

-   `get_artist_genres_batch`: Mengambil genre untuk banyak artis sekaligus secara efisien.
-   `scrape_spotify_custom_format`: Mengambil semua lagu dari playlist, mengumpulkan data, dan memformatnya sesuai kebutuhan.

In [6]:
def get_artist_genres_batch(artist_ids):
    """
    Mengambil genre untuk banyak artis sekaligus (Batching) agar scraping cepat.
    Maksimal 50 ID per request menurut aturan Spotify.
    """
    artist_genres = {}
    # Pecah list ID menjadi chunk berukuran 50
    for i in range(0, len(artist_ids), 50):
        chunk = artist_ids[i:i+50]
        artists_info = sp.artists(chunk)

        for artist in artists_info['artists']:
            # Ambil genre pertama jika ada, atau 'Pop' sebagai default jika kosong
            if artist['genres']:
                # Mengambil genre utama dan membuatnya Title Case (misal: "indonesian pop" -> "Indonesian Pop")
                main_genre = artist['genres'][0].title()
            else:
                main_genre = "Pop" # Fallback jika artis tidak punya genre spesifik

            artist_genres[artist['id']] = main_genre

    return artist_genres

### 3. Eksekusi Program dan Penyimpanan Data

Bagian ini mengatur playlist yang akan di-scrape, memanggil fungsi scraping, menggabungkan hasil, dan menyimpannya ke dalam file CSV. Anda bisa mengganti link playlist sesuai kebutuhan Anda.

In [5]:
# Ganti link di bawah dengan link playlist kamu yang asli
link_90an = "https://open.spotify.com/playlist/7ihAeoNOQDXoKbN6zdGX3q?si=1c2ae78531e544f9"
link_2020an = "https://open.spotify.com/playlist/3dBChWwvtggOLWpftW85Cd?si=88bc99a18e704ce4"

print("--- START SCRAPING ---")

# Proses Playlist
data_90 = scrape_spotify_custom_format(link_90an, '90an')
data_20 = scrape_spotify_custom_format(link_2020an, '2020an')

# Gabungkan Data
all_data = data_90 + data_20

# Buat DataFrame
df = pd.DataFrame(all_data)

# Urutan kolom dipastikan sesuai request
desired_columns = ['Judul', 'Artis', 'Tahun', 'Era', 'Genre', 'Penulis', 'Lirik']
df = df[desired_columns]

# Simpan ke CSV
filename = 'dataset_lagu_final.csv'
df.to_csv(filename, index=False)

print("\n" + "="*40)
print(f"SELESAI! {len(df)} lagu berhasil disimpan ke '{filename}'")
print("Contoh 5 data teratas:")
print(df.head())
print("="*40)

--- START SCRAPING ---
Sedang membaca playlist 90an...
Mengambil data genre untuk 39 artis...
Sedang membaca playlist 2020an...
Mengambil data genre untuk 33 artis...

SELESAI! 251 lagu berhasil disimpan ke 'dataset_lagu_final.csv'
Contoh 5 data teratas:
         Judul         Artis Tahun   Era            Genre       Penulis Lirik
0       Dan...   Sheila On 7  1999  90an   Indonesian Pop   Sheila On 7      
1  Aku Milikmu       Dewa 19  1994  90an   Indonesian Pop       Dewa 19      
2   Matahariku  Nike Ardilla  1991  90an  Indonesian Rock  Nike Ardilla      
3       Memang         Slank  1990  90an  Indonesian Rock         Slank      
4        Sobat          Padi  1999  90an   Indonesian Pop          Padi      
